In [1]:
import pandas as pd
import pickle
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
from downcast import reduce
from numba import jit
import time
import gc
from downcast import reduce
import random
import warnings
from math import ceil
from tqdm import trange
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore")

In [2]:
# Large dataset, use 2 methods to save memory
@jit
def reduce_mem_usage(df, verbose=True, method='downcast'):
    start = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    if method == 'downcast':
        df = reduce(df)
    else:
        for col in df.columns:
            col_type = df[col].dtypes
            if col_type in numerics:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction) and time is '.format(end_mem, 100 * (
                    start_mem - end_mem) / start_mem) + str(time.time() - start))
    return df

In [3]:
# Merging by concat to not lose dtypes
def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

In [4]:
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [5]:
#Read data
calendar = pd.read_csv('D:/HKUST/MAFS6010Z AI in Fintech/project3/calendar.csv')
calendar['date'] = pd.to_datetime(calendar['date'])
sell_prices = pd.read_csv('D:/HKUST/MAFS6010Z AI in Fintech/project3/sell_prices.csv')
sell_prices = reduce_mem_usage(sell_prices)
sales_train = pd.read_csv('D:/HKUST/MAFS6010Z AI in Fintech/project3/sales_train_evaluation.csv')
sales_train = reduce_mem_usage(sales_train)

Mem. usage decreased to 45.76 Mb (78.1% reduction) and time is 0.8527185916900635
Mem. usage decreased to 96.30 Mb (78.7% reduction) and time is 128.18330121040344


In [6]:
calendar = calendar.fillna('no_event')
calendar = reduce_mem_usage(calendar)

Mem. usage decreased to  0.13 Mb (40.4% reduction) and time is 0.011980772018432617


In [7]:
TARGET = 'sales'         # Our main target
END_TRAIN = 1941         # Last day in train set
MAIN_INDEX = ['id','d']  # We can identify item by these columns

In [8]:
########################### Make Grid
#################################################################################
print('Create Grid')

# We can tranform horizontal representation representation 바꾸기
# to vertical "view"
# Our "index" will be 'id','item_id','dept_id','cat_id','store_id','state_id'
# and labels are 'd_' coulmns

index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
grid_df = pd.melt(sales_train, id_vars = index_columns, var_name = 'd', value_name = TARGET)

# If we look on train_df we se that 
# we don't have a lot of traning rows
# but each day can provide more train data
print('Train rows:', len(sales_train), len(grid_df))

# To be able to make predictions
# we need to add "test set" to our grid
add_grid = pd.DataFrame()
for i in trange(1,29):
    temp_df = sales_train[index_columns]
    temp_df = temp_df.drop_duplicates()
    temp_df['d'] = 'd_'+ str(END_TRAIN+i)
    temp_df[TARGET] = np.nan
    add_grid = pd.concat([add_grid,temp_df])

grid_df = pd.concat([grid_df,add_grid])
grid_df = grid_df.reset_index(drop=True)

# Remove some temoprary DFs
del temp_df, add_grid
gc.collect()

# We will not need original train_df
# anymore and can remove it
# del train_df

# Let's check our memory usage
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

# We can free some memory 
# by converting "strings" to categorical
# it will not affect merging and 
# we will not lose any valuable data
for col in index_columns:
    grid_df[col] = grid_df[col].astype('category')

# Let's check again memory usage
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

Create Grid
Train rows: 30490 59181090


100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 34.15it/s]


    Original grid_df:   1.3GiB
     Reduced grid_df:   1.3GiB


In [9]:
%%time
########################### Product Release date
#################################################################################

print('Release week')

# It seems that leadings zero values
# in each train_df item row
# are not real 0 sales but mean
# absence for the item in the store
# we can safe some memory by removing
# such zeros

# Prices are set by week
# so it we will have not very accurate release week 
release_df = sell_prices.groupby(['store_id','item_id'])['wm_yr_wk'].agg(['min']).reset_index()
release_df.columns = ['store_id','item_id','release']

# Now we can merge release_df
grid_df = merge_by_concat(grid_df, release_df, ['store_id','item_id'])
del release_df

# We want to remove some "zeros" rows
# from grid_df 
# to do it we need wm_yr_wk column
# let's merge partly calendar_df to have it
grid_df = merge_by_concat(grid_df, calendar[['wm_yr_wk','d']], ['d'])
                      
# Now we can cutoff some rows 
# and safe memory 
grid_df = grid_df[grid_df['wm_yr_wk']>=grid_df['release']]
grid_df = grid_df.reset_index(drop=True)

# Let's check our memory usage
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

# Should we keep release week 
# as one of the features?
# Only good CV can give the answer.
# Let's minify the release values.
# Min transformation will not help here 
# as int16 -> Integer (-32768 to 32767)
# and our grid_df['release'].max() serves for int16
# but we have have an idea how to transform 
# other columns in case we will need it
grid_df['release'] = grid_df['release'] - grid_df['release'].min()
grid_df['release'] = grid_df['release'].astype(np.int16)

grid_df['daily_avg_sold'] = grid_df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id','d'])[TARGET].transform('mean').astype(np.float16)
grid_df['avg_sold'] = grid_df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])[TARGET].transform('mean').astype(np.float16)
grid_df['selling_trend'] = (grid_df['daily_avg_sold'] - grid_df['avg_sold']).astype(np.float16)
grid_df = grid_df.drop(['daily_avg_sold','avg_sold'], axis=1)


# Let's check again memory usage
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

Release week
    Original grid_df:   1.2GiB
     Reduced grid_df:   1.3GiB
Wall time: 56.3 s


In [10]:
########################## Save part 1
#################################################################################
print('Save Part 1')

# We have our BASE grid ready
# and can save it as pickle file
# for future use (model training)
grid_df = reduce_mem_usage(grid_df)
grid_df.to_pickle('E:/grid_part_1.pkl')

print('Size:', grid_df.shape)

Save Part 1
Mem. usage decreased to 820.84 Mb (40.0% reduction) and time is 3.739992618560791
Size: (47735397, 11)


In [11]:
%%time
########################### Prices
#################################################################################

print('Prices')

# We can do some basic aggregations
sell_prices['price_max'] = sell_prices.groupby(['store_id','item_id'])['sell_price'].transform('max')
sell_prices['price_min'] = sell_prices.groupby(['store_id','item_id'])['sell_price'].transform('min')
sell_prices['price_std'] = sell_prices.groupby(['store_id','item_id'])['sell_price'].transform('std')
sell_prices['price_mean'] = sell_prices.groupby(['store_id','item_id'])['sell_price'].transform('mean')

# and do price normalization (min/max scaling)
sell_prices['price_norm'] = sell_prices['sell_price']/sell_prices['price_max']

# Some items are can be inflation dependent
# and some items are very "stable"

sell_prices['price_nunique'] = sell_prices.groupby(['store_id','item_id'])['sell_price'].transform('nunique') 
sell_prices['item_nunique'] = sell_prices.groupby(['store_id','sell_price'])['item_id'].transform('nunique')

# I would like some "rolling" aggregations
# but would like months and years as "window"
calendar_prices = calendar[['wm_yr_wk','month','year']]
calendar_prices = calendar_prices.drop_duplicates(subset=['wm_yr_wk']) # distinct(.keep_all = True)
sell_prices = sell_prices.merge(calendar_prices[['wm_yr_wk','month','year']], on=['wm_yr_wk'], how='left')
del calendar_prices
gc.collect()

# Now we can add price "momentum" (some sort of)
# Shifted by week 
# by month mean
# by year mean
sell_prices['price_momentum'] = sell_prices['sell_price']/sell_prices.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))
sell_prices['price_momentum_m'] = sell_prices['sell_price']/sell_prices.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
sell_prices['price_momentum_y'] = sell_prices['sell_price']/sell_prices.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')

sell_prices.drop(columns=['month', 'year'])

Prices
Wall time: 10.9 s


,store_id,item_id,wm_yr_wk,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique,price_momentum,price_momentum_m,price_momentum_y
0,CA_1,HOBBIES_1_001,11325,9.578125,9.578125,8.257812,0.152402,8.28125,1.000000,3,3,NaN,1.126953,1.144531
1,CA_1,HOBBIES_1_001,11326,9.578125,9.578125,8.257812,0.152402,8.28125,1.000000,3,3,1.000000,1.126953,1.144531
2,CA_1,HOBBIES_1_001,11327,8.257812,9.578125,8.257812,0.152402,8.28125,0.862305,3,5,0.862305,0.971680,0.986816
3,CA_1,HOBBIES_1_001,11328,8.257812,9.578125,8.257812,0.152402,8.28125,0.862305,3,5,1.000000,1.000000,0.986816
4,CA_1,HOBBIES_1_001,11329,8.257812,9.578125,8.257812,0.152402,8.28125,0.862305,3,5,1.000000,1.000000,0.986816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.000000,1.000000,1.000000,0.000000,1.00000,1.000000,1,142,1.000000,1.000000,1.000000
6841117,WI_3,FOODS_3_827,11618,1.000000,1.000000,1.000000,0.000000,1.00000,1.000000,1,142,1.000000,1.000000,1.000000
6841118,WI_3,FOODS_3_827,11619,1.000000,1.000000,1.000000,0.000000,1.00000,1.000000,1,142,1.000000,1.000000,1.000000
6841119,WI_3,FOODS_3_827,11620,1.000000,1.000000,1.000000,0.000000,1.00000,1.000000,1,142,1.000000,1.000000,1.000000


In [12]:
grid_df = reduce_mem_usage(grid_df)
sell_prices = reduce_mem_usage(sell_prices)

Mem. usage decreased to 820.84 Mb (0.0% reduction) and time is 1.2277166843414307
Mem. usage decreased to 241.48 Mb (33.9% reduction) and time is 0.6084496974945068


In [13]:
########################### Merge prices and save part 2
#################################################################################
print('Merge prices and save part 2')

# Merge Prices
original_columns = list(grid_df)
grid_df = grid_df.merge(sell_prices, on=['store_id','item_id','wm_yr_wk'], how='left')
keep_columns = [col for col in list(grid_df) if col not in original_columns]
grid_df = grid_df[MAIN_INDEX+keep_columns]
grid_df = reduce_mem_usage(grid_df)

# Safe part 2
grid_df.to_pickle('grid_part_2.pkl')
print('Size:', grid_df.shape)

Merge prices and save part 2
Mem. usage decreased to 1640.18 Mb (0.0% reduction) and time is 3.729440927505493
Size: (47735397, 15)


In [18]:
# We don't need prices_df anymore
del grid_df
gc.collect()

# We can remove new columns
# or just load part_1
grid_df = pd.read_pickle('E:/grid_part_1.pkl')

In [19]:
%%time
########################### Merge calendar
#################################################################################

grid_df = grid_df[MAIN_INDEX]

# Merge calendar partly
icols = ['date',
         'd',
         'event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI']

grid_df = grid_df.merge(calendar[icols], on=['d'], how='left')

# Minify data
# 'snap_' columns we can convert to bool or int8
icols = ['event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI']
for col in icols:
    grid_df[col] = grid_df[col].astype('category')

# Convert to DateTime
# grid_df['date'] = pd.to_datetime(grid_df['date'])

# Make some features from date
grid_df['tm_d'] = grid_df['date'].dt.day.astype(np.int8)
grid_df['tm_w'] = grid_df['date'].dt.week.astype(np.int8)
grid_df['tm_m'] = grid_df['date'].dt.month.astype(np.int8)
grid_df['tm_y'] = grid_df['date'].dt.year
grid_df['tm_y'] = (grid_df['tm_y'] - grid_df['tm_y'].min()).astype(np.int8)
grid_df['tm_wm'] = grid_df['tm_d'].apply(lambda x: ceil(x/7)).astype(np.int8)

grid_df['tm_dw'] = grid_df['date'].dt.dayofweek.astype(np.int8)
grid_df['tm_w_end'] = (grid_df['tm_dw']>=5).astype(np.int8)

# Remove date
del grid_df['date']
gc.collect()

Wall time: 49.9 s


49

In [20]:
########################### Save part 3 (Dates)
#################################################################################
print('Save part 3')

# Safe part 3
grid_df = reduce_mem_usage(grid_df)
grid_df.to_pickle('grid_part_3.pkl')
print('Size:', grid_df.shape)

# We don't need calendar_df anymore
del calendar
del grid_df
gc.collect()

Save part 3
Mem. usage decreased to 1183.87 Mb (0.0% reduction) and time is 0.5964627265930176
Size: (47735397, 16)


24

In [22]:
########################### Some additional cleaning
#################################################################################

## Part 1
# Convert 'd' to int
grid_df = pd.read_pickle('E:/grid_part_1.pkl')
grid_df['d'] = grid_df['d'].apply(lambda x: x[2:]).astype(np.int16)

# Remove 'wm_yr_wk'
# as test values are not in train set
del grid_df['wm_yr_wk']
grid_df = reduce_mem_usage(grid_df)
grid_df.to_pickle('E:/grid_part_1.pkl')

del grid_df
gc.collect()

Mem. usage decreased to 728.64 Mb (0.0% reduction) and time is 1.1877429485321045


40

In [23]:
grid1 = pd.read_pickle('E:/grid_part_1.pkl')
grid1

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,selling_trend
0,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,1,12.0,0,4.714844
1,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,1,2.0,0,0.821289
2,HOBBIES_1_010_CA_1_evaluation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,1,0.0,0,-0.716797
3,HOBBIES_1_012_CA_1_evaluation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,1,0.0,0,-0.392090
4,HOBBIES_1_015_CA_1_evaluation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,1,4.0,0,-2.015625
...,...,...,...,...,...,...,...,...,...,...
47735392,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,1969,NaN,0,NaN
47735393,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,1969,NaN,0,NaN
47735394,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,1969,NaN,0,NaN
47735395,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,1969,NaN,230,NaN


In [24]:
grid2 = pd.read_pickle('E:/grid_part_2.pkl')
grid2

,id,d,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique,month,year,price_momentum,price_momentum_m,price_momentum_y
0,HOBBIES_1_008_CA_1_evaluation,d_1,0.459961,0.500000,0.419922,0.019791,0.476318,0.919922,4,16,1,2011,NaN,0.968750,0.949707
1,HOBBIES_1_009_CA_1_evaluation,d_1,1.559570,1.769531,1.559570,0.032715,1.764648,0.881348,2,9,1,2011,NaN,0.885742,0.896484
2,HOBBIES_1_010_CA_1_evaluation,d_1,3.169922,3.169922,2.970703,0.046173,2.982422,1.000000,2,20,1,2011,NaN,1.064453,1.043945
3,HOBBIES_1_012_CA_1_evaluation,d_1,5.980469,6.519531,5.980469,0.115906,6.468750,0.917480,3,71,1,2011,NaN,0.922363,0.959473
4,HOBBIES_1_015_CA_1_evaluation,d_1,0.700195,0.720215,0.680176,0.011353,0.707031,0.972168,3,16,1,2011,NaN,0.990234,1.001953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47735392,FOODS_3_823_WI_3_evaluation,d_1969,2.980469,2.980469,2.480469,0.171875,2.802734,1.000000,5,206,6,2016,1.0,1.032227,1.022461
47735393,FOODS_3_824_WI_3_evaluation,d_1969,2.480469,2.679688,2.000000,0.252930,2.507812,0.925781,4,135,6,2016,1.0,0.985840,1.112305
47735394,FOODS_3_825_WI_3_evaluation,d_1969,3.980469,4.378906,3.980469,0.187866,4.117188,0.909180,3,150,6,2016,1.0,0.957520,1.000000
47735395,FOODS_3_826_WI_3_evaluation,d_1969,1.280273,1.280273,1.280273,0.000000,1.280273,1.000000,1,44,6,2016,1.0,1.000000,1.000000


In [25]:
grid3 = pd.read_pickle('E:/grid_part_3.pkl')
grid3

,id,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end
0,HOBBIES_1_008_CA_1_evaluation,d_1,no_event,no_event,no_event,no_event,0,0,0,29,4,1,0,5,5,1
1,HOBBIES_1_009_CA_1_evaluation,d_1,no_event,no_event,no_event,no_event,0,0,0,29,4,1,0,5,5,1
2,HOBBIES_1_010_CA_1_evaluation,d_1,no_event,no_event,no_event,no_event,0,0,0,29,4,1,0,5,5,1
3,HOBBIES_1_012_CA_1_evaluation,d_1,no_event,no_event,no_event,no_event,0,0,0,29,4,1,0,5,5,1
4,HOBBIES_1_015_CA_1_evaluation,d_1,no_event,no_event,no_event,no_event,0,0,0,29,4,1,0,5,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47735392,FOODS_3_823_WI_3_evaluation,d_1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,19,24,6,5,3,6,1
47735393,FOODS_3_824_WI_3_evaluation,d_1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,19,24,6,5,3,6,1
47735394,FOODS_3_825_WI_3_evaluation,d_1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,19,24,6,5,3,6,1
47735395,FOODS_3_826_WI_3_evaluation,d_1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,19,24,6,5,3,6,1


In [26]:
del grid1, grid2, grid3
gc.collect()

grid_df = pd.read_pickle('E:/grid_part_1.pkl')

# We need only 'id','d','sales'
# to make lags and rollings
grid_df = grid_df[['id','d','sales']]
SHIFT_DAY = 28
start_time = time.time()
print('Create lags')

LAG_DAYS = [col for col in range(SHIFT_DAY, SHIFT_DAY + 15)]
grid_df = grid_df.assign(**{
    '{}_lag_{}'.format(col, l): grid_df.groupby(['id'])[col].transform(lambda x: x.shift(l))
    for l in LAG_DAYS
    for col in [TARGET]
})

# Minify lag columns
for col in list(grid_df):
    if 'lag' in col:
        grid_df[col] = grid_df[col].astype(np.float16)

print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

Create lags
3.71 min: Lags


In [27]:
grid_df

,id,d,sales,sales_lag_28,sales_lag_29,sales_lag_30,sales_lag_31,sales_lag_32,sales_lag_33,sales_lag_34,sales_lag_35,sales_lag_36,sales_lag_37,sales_lag_38,sales_lag_39,sales_lag_40,sales_lag_41,sales_lag_42
0,HOBBIES_1_008_CA_1_evaluation,1,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_009_CA_1_evaluation,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_010_CA_1_evaluation,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_012_CA_1_evaluation,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_015_CA_1_evaluation,1,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47735392,FOODS_3_823_WI_3_evaluation,1969,NaN,1.0,1.0,0.0,0.0,1.0,1.0,0.0,3.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
47735393,FOODS_3_824_WI_3_evaluation,1969,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47735394,FOODS_3_825_WI_3_evaluation,1969,NaN,2.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0,1.0,3.0,3.0,0.0,1.0
47735395,FOODS_3_826_WI_3_evaluation,1969,NaN,0.0,1.0,1.0,1.0,0.0,6.0,4.0,1.0,1.0,1.0,1.0,2.0,0.0,1.0,1.0


In [28]:
# Rollings
# with 28 day shift
start_time = time.time()
print('Create rolling aggs')

for i in [7,14,30,60]:
    time1 = time.time()
    print('Rolling period:', i)
    grid_df['rolling_mean_'+str(i)] = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).mean()).astype(np.float16)
    grid_df['rolling_std_'+str(i)]  = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).std()).astype(np.float16)
    print('Finished rolling period ' +  str(i) + ' and time is {}s'.format(str(time.time() - time1)))

# Rollings
# with sliding shift
for d_shift in [1,7,14]: 
    part_time = time.time()
    print('Shifting period:', d_shift)
    for d_window in [7,14,30,60]:
        time2 = time.time()
        col_name = 'rolling_mean_tmp_'+str(d_shift)+'_'+str(d_window)
        grid_df[col_name] = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(d_shift).rolling(d_window).mean()).astype(np.float16)
        print('Finished Shifting window ' +  str(d_window) + ' and time is {}s'.format(str(time.time() - time2)))
    print('Finished Shifting period ' +  str(d_shift) + ' and time is {}s'.format(str(time.time() - part_time)))
    
    
print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

Create rolling aggs
Rolling period: 7
Finished rolling period 7 and time is 43.611454248428345s
Rolling period: 14
Finished rolling period 14 and time is 43.14370918273926s
Rolling period: 30
Finished rolling period 30 and time is 44.634148836135864s
Rolling period: 60
Finished rolling period 60 and time is 43.92546629905701s
Shifting period: 1
Finished Shifting window 7 and time is 21.013791799545288s
Finished Shifting window 14 and time is 21.634317636489868s
Finished Shifting window 30 and time is 21.511520862579346s
Finished Shifting window 60 and time is 21.81537365913391s
Finished Shifting period 1 and time is 85.97600150108337s
Shifting period: 7
Finished Shifting window 7 and time is 21.719974756240845s
Finished Shifting window 14 and time is 21.671499252319336s
Finished Shifting window 30 and time is 21.36288547515869s
Finished Shifting window 60 and time is 21.589792251586914s
Finished Shifting period 7 and time is 86.34614491462708s
Shifting period: 14
Finished Shifting wind

In [29]:
grid_df

,id,d,sales,sales_lag_28,sales_lag_29,sales_lag_30,sales_lag_31,sales_lag_32,sales_lag_33,sales_lag_34,...,rolling_mean_tmp_1_30,rolling_mean_tmp_1_60,rolling_mean_tmp_7_7,rolling_mean_tmp_7_14,rolling_mean_tmp_7_30,rolling_mean_tmp_7_60,rolling_mean_tmp_14_7,rolling_mean_tmp_14_14,rolling_mean_tmp_14_30,rolling_mean_tmp_14_60
0,HOBBIES_1_008_CA_1_evaluation,1,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_009_CA_1_evaluation,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_010_CA_1_evaluation,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_012_CA_1_evaluation,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_015_CA_1_evaluation,1,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47735392,FOODS_3_823_WI_3_evaluation,1969,NaN,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47735393,FOODS_3_824_WI_3_evaluation,1969,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47735394,FOODS_3_825_WI_3_evaluation,1969,NaN,2.0,0.0,1.0,0.0,1.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47735395,FOODS_3_826_WI_3_evaluation,1969,NaN,0.0,1.0,1.0,1.0,0.0,6.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
########################## Export
#################################################################################
print('Save lags and rollings')
grid_df = reduce_mem_usage(grid_df)
grid_df.to_pickle('lags_df_'+str(SHIFT_DAY)+'.pkl')

Save lags and rollings
Mem. usage decreased to 3461.07 Mb (0.0% reduction) and time is 14.169179201126099


In [32]:
%%time
########################### Apply on grid_df
#################################################################################
# lets read grid from 
# https://www.kaggle.com/kyakovlev/m5-simple-fe
# to be sure that our grids are aligned by index

grid_df = pd.read_pickle('E:/grid_part_1.pkl')
grid_df['sales'][grid_df['d']>(1941-28)] = np.nan
base_cols = list(grid_df)

icols =  [
            ['state_id'],
            ['store_id'],
            ['cat_id'],
            ['dept_id'],
            ['state_id', 'cat_id'],
            ['state_id', 'dept_id'],
            ['store_id', 'cat_id'],
            ['store_id', 'dept_id'],
            ['item_id'],
            ['item_id', 'state_id'],
            ['item_id', 'store_id']
            ]

for col in icols:
    print('Encoding', col)
    start = time.time()
    col_name = '_'+'_'.join(col)+'_'
    grid_df['enc'+col_name+'mean'] = grid_df.groupby(col)['sales'].transform('mean').astype(np.float16)
    grid_df['enc'+col_name+'std'] = grid_df.groupby(col)['sales'].transform('std').astype(np.float16)
    print('Finished encoding ' + str(col) + ' and time is {}s'.format(str(time.time()-start)))

keep_cols = [col for col in list(grid_df) if col not in base_cols]
grid_df = grid_df[['id','d']+keep_cols]

Encoding ['state_id']
Finished encoding ['state_id'] and time is 2.0155882835388184s
Encoding ['store_id']
Finished encoding ['store_id'] and time is 1.996659517288208s
Encoding ['cat_id']
Finished encoding ['cat_id'] and time is 1.9827768802642822s
Encoding ['dept_id']
Finished encoding ['dept_id'] and time is 2.0055975914001465s
Encoding ['state_id', 'cat_id']
Finished encoding ['state_id', 'cat_id'] and time is 4.276397228240967s
Encoding ['state_id', 'dept_id']
Finished encoding ['state_id', 'dept_id'] and time is 4.335480690002441s
Encoding ['store_id', 'cat_id']
Finished encoding ['store_id', 'cat_id'] and time is 4.468971014022827s
Encoding ['store_id', 'dept_id']
Finished encoding ['store_id', 'dept_id'] and time is 4.568785667419434s
Encoding ['item_id']
Finished encoding ['item_id'] and time is 1.9108881950378418s
Encoding ['item_id', 'state_id']
Finished encoding ['item_id', 'state_id'] and time is 5.076319456100464s
Encoding ['item_id', 'store_id']
Finished encoding ['item_

In [33]:
print('Save Mean/Std encoding')
grid_df = reduce_mem_usage(grid_df)
grid_df.to_pickle('mean_encoding_df.pkl')

Save Mean/Std encoding
Mem. usage decreased to 2185.39 Mb (0.0% reduction) and time is 8.074897289276123
